# CyprusDB generation

This notebook contains all the steps followed to generate the CyprusDB dataset. This ensures its reproducibility.

## Data sources

1) [CyStat's 2011 census](https://www.data.gov.cy/dataset/%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CF%8C%CF%82-%CE%BA%CE%B1%CF%84%CE%AC-%CF%84%CF%8C%CF%80%CE%BF-%CE%B4%CE%B9%CE%B1%CE%BC%CE%BF%CE%BD%CE%AE%CF%82-%CE%B1%CF%80%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AE-%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CE%BF%CF%8D-2011)
2) [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview?hl=en-419)

# Preparation

In [1]:
### Imports ###

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import folium

# Data retrieval
import requests
import json

# Others
import time
from pprint import pprint
import os
import glob
from datetime import date

# 1) Population list

Source: [CyStat](https://www.data.gov.cy/dataset/%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CF%8C%CF%82-%CE%BA%CE%B1%CF%84%CE%AC-%CF%84%CF%8C%CF%80%CE%BF-%CE%B4%CE%B9%CE%B1%CE%BC%CE%BF%CE%BD%CE%AE%CF%82-%CE%B1%CF%80%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AE-%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CE%BF%CF%8D-2011)

The base town list is extracted from the above link. The list is then cleaned and the population is extracted from the excel file. The complete list of settlements used in this project is teken from the sheet `Γ2`.

In [2]:
# Open raw file
filepath = 'sources/Census 2011 Excel format/POP_CEN_11-POP_PLACE_RESID-EL-171115.xls'

# Read file
raw_census_df = pd.read_excel(filepath, sheet_name='Γ2', skiprows=3)
raw_census_df.head()

,Unnamed: 0,ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ,"ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ",ΝΟΙΚΟΚΥΡΙΑ,Unnamed: 4,Unnamed: 5,Unnamed: 6,ΙΔΡΥΜΑΤΑ,Unnamed: 8,Unnamed: 9,Unnamed: 10,ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,ΑΡΙΘΜΟΣ,ΠΛΗΘΥΣΜΟΣ,NaN,NaN,ΑΡΙΘΜΟΣ,ΠΛΗΘΥΣΜΟΣ,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Σύνολο,Άνδρες,Γυναίκες,NaN,Σύνολο,Άνδρες,Γυναίκες,Σύνολο,Άνδρες,Γυναίκες
2,NaN,NaN,Σύνολο,303242,836566,407228,429338,211,3841,1552,2289,840407,408780,431627
3,NaN,1,Επαρχία Λευκωσίας,119203,324952,157307,167645,94,2028,955,1073,326980,158262,168718
4,NaN,1000,Δήμος Λευκωσίας,22833,54452,26086,28366,11,562,434,128,55014,26520,28494


## Data cleaning

In [3]:
# Retain only relevant columns
to_retain = [
    'ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ',
    'ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ',
    'ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ',
    'Unnamed: 12', # Males
    'Unnamed: 13' # Fermales
]

census_df = raw_census_df[to_retain].copy()

# Rename columns
to_rename = {
    'ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ': 'geo_code',
    'ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ': 'town',
    'ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ': 'population',
    'Unnamed: 12' : 'male_population',
    'Unnamed: 13' : 'female_population'
}

census_df.rename(columns=to_rename, inplace=True)

# Drop the three first rows for format purposes
census_df.drop([0, 1, 2], inplace=True)

# Drop the last six rows, which provide no information
census_df.drop(census_df.tail(6).index, inplace=True)

# Fill NaNs with 0 in male and female population columns
census_df.fillna(value = {'male_population' : 0, 'female_population' : 0}, inplace=True)

census_df.head()

,geo_code,town,population,male_population,female_population
3,1,Επαρχία Λευκωσίας,326980,158262,168718
4,1000,Δήμος Λευκωσίας,55014,26520,28494
5,100001,Άγιος Ανδρέας,5767,2817,2950
6,100002,Τρυπιώτης,2158,983,1175
7,100003,Νεμπέτ Χανέ,189,86,103


### Create the 'district' column

In [4]:
# Extract the district from the 'town' column and create a new column
# if 'Επαρχία' is in the town name, then the district is the town name
census_df['district'] = census_df['town'].apply(lambda x: x.split(' ')[1] if 'Επαρχία' in x else np.nan)

# Fill the NaN values with the previous value
# using a forward fill
census_df['district'].fillna(method='ffill', inplace=True)

# Remove the rows that contain the district name
census_df = census_df[~census_df['town'].str.contains('Επαρχία')]

census_df.head()

,geo_code,town,population,male_population,female_population,district
4,1000,Δήμος Λευκωσίας,55014,26520,28494,Λευκωσίας
5,100001,Άγιος Ανδρέας,5767,2817,2950,Λευκωσίας
6,100002,Τρυπιώτης,2158,983,1175,Λευκωσίας
7,100003,Νεμπέτ Χανέ,189,86,103,Λευκωσίας
8,100004,Ταμπάκ Χανέ,299,117,182,Λευκωσίας


### Include suburbs into the main cities
This requieres reversing the genitives of the dimos to the nominative case. Since the number of suburbs is small, we can do this manually


In [5]:
# Map genitive town names to nominative town names
dimos_manual_mapping = {
    # Nicosia
    'Δήμος Λευκωσίας' : 'Λευκωσία',
    'Δήμος Αγίου Δομετίου' : 'Άγιος Δομέτιος',
    'Δήμος Έγκωμης' : 'Έγκωμη',
    'Δήμος Στροβόλου' : 'Στροβόλος',
    'Δήμος Αγλαντζιάς' : 'Αγλαντζιά',
    'Δήμος Λακατάμειας' : 'Λακατάμεια',
    'Δήμος Λατσιών' : 'Λατσιά',
    'Δήμος Ιδαλίου' : 'Δάλι',
    # Larnaka
    'Δήμος Λάρνακας' : 'Λάρνακα',
    'Δήμος Αραδίππου' : 'Αραδίππου',
    # Limassol
    'Δήμος Λεμεσού' : 'Λεμεσός',
    'Δήμος Μέσα Γειτονιάς' : 'Μέσα Γειτονιά',
    'Δήμος Αγίου Αθανασίου' : 'Άγιος Αθανάσιος',
    'Δήμος Γερμασόγειας' : 'Γερμασόγεια',
    'Δήμος Κάτω Πολεμιδιών' : 'Κάτω Πολεμίδια',
    # Paphos
    'Δήμος Πάφου' : 'Πάφος',
    'Δήμος Γεροσκήπου' : 'Γεροσκήπου',
    'Δήμος Πόλεως Χρυσοχούς' : 'Πόλις Χρυσοχούς',
    'Δήμος Πέγειας' : 'Πέγεια'
}

# Create a column indicating whether the town name is a dimos
census_df['is_dimos'] = census_df['town'].apply(lambda x: True if x in dimos_manual_mapping.keys() else False)

# Replace dimos names with settlement names
census_df['town'] = census_df['town'].apply(lambda x: dimos_manual_mapping[x] if x in dimos_manual_mapping.keys() else x)

In [6]:
census_df.head(5)

,geo_code,town,population,male_population,female_population,district,is_dimos
4,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
5,100001,Άγιος Ανδρέας,5767,2817,2950,Λευκωσίας,False
6,100002,Τρυπιώτης,2158,983,1175,Λευκωσίας,False
7,100003,Νεμπέτ Χανέ,189,86,103,Λευκωσίας,False
8,100004,Ταμπάκ Χανέ,299,117,182,Λευκωσίας,False


## Remove suburbs from locations

This is done for several reasons:
- Have a consistent data structure: all locations are settlements, not suburbs
- Avoid double population counts

This behaviour can be switched off by setting `remove_suburbs=False`

In [7]:
remove_suburbs = True

# Remove suburbs
# The suburbs are the towns that have a six-digit geo code
if remove_suburbs:
    census_df = census_df[census_df['geo_code'].apply(lambda x: len(str(x)) < 6)].reset_index(drop=True)

In [9]:
census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True


In [10]:
# Save a checkpoint of the dataframe to a csv file
version = 1

census_df.to_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv', index=False)

In [17]:
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

# 2) Google Maps API (Geocoding API)

Source: [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview?hl=en-419)

## Notes:
- The API key is stored in a file called `api_key.txt`. This file is not included in the repository. To reproduce the results here, you need to create your own API key and store it in a file called `api_key.txt` in the folder `private_utils`.
- Requests are limited to 2500 per day. 
- Retrieving data from the API is not free. Be aware of the costs when rebuilding the dataset.

In [12]:
# Read API key
with open('private_utils/api_key.txt', 'r') as f:
    api_key = f.read()

## Auxiliary functions

In [13]:
# Extract coordinates for towns in Cyprus from Geocoding API
def extract_coordinates(
        town: str,
        district: str,
        api_key: str = api_key):
    
    # Set search term
    query = town + ' ' + district + ' Κύπρος'

    # Set Geocoding API URL
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + '&key=' + api_key

    # Extract coordinates
    response = requests.get(url)
    data = json.loads(response.text)
    
    # Extract coordinates
    lat = data['results'][0]['geometry']['location']['lat']
    lng = data['results'][0]['geometry']['location']['lng']

    return lat, lng

## Data retrieval

In [18]:
census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True


In [24]:
# Generate coordinates if requested
generate_coordinates = False

if generate_coordinates:
    # Extract coordinates for towns in Cyprus from Geocoding API
    # Takes ~ 1 minute
    census_df['lat'], census_df['lon'] = zip(*census_df.apply(lambda x: extract_coordinates(x['town'], x['district']), axis=1))

    # Save coordinates with retrieval date
    census_coordinates = census_df[['town', 'district', 'lat', 'lon']]
    census_coordinates.to_csv(f'sources/Geocoding API/geocoding_coordinates_{date.today().strftime("%Y-%m-%d")}.csv', index=False)

else:
    # Select the latest coordinates file
    list_of_files = glob.glob('sources/Geocoding API/*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # Load coordinates from file
    census_coordinates = pd.read_csv(latest_file)

# Add coordinates to census dataframe
# census_df = census_df.merge(census_coordinates, on=['town', 'district'], how='left')
census_df[['lat', 'lon']] = census_coordinates[['lat', 'lon']]

census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos,lat,lon
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True,35.185566,33.382276
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True,35.172787,33.329092
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True,35.153823,33.316954
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True,35.143663,33.343791
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True,35.149803,33.394086


## Plot and inspect results

In [25]:
# Show the retrieval results to check for errors
# Plot all towns in Cyprus
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df)):
    folium.CircleMarker(
        location=[census_df['lat'][i], census_df['lon'][i]],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

## Results notes
1) Πρωταράς (Protarás) in Ammochostos district is not registered in the census
2) The coordinates for the town of Πόλη Χρυσοχούς (Póli Chrysochoús) are not correct. This is due to conflicting names with other natural places and locations. Since it is the only instance, a manual correction is applied.

In [31]:
# Prepare manual corrections 
coordinates_manual_mapping = {
    'Πόλις Χρυσοχούς' : [35.0339441, 32.4253751]
}

# Apply manual corrections
for town in coordinates_manual_mapping.keys():
    census_df.loc[census_df['town'] == town, ['lat', 'lon']] = coordinates_manual_mapping[town]

## Inspect results after correction

In [33]:
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df)):
    folium.CircleMarker(
        location=[census_df['lat'][i], census_df['lon'][i]],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

In [34]:
# Save a checkpoint of the dataframe to a csv file
version = 2

census_df.to_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv', index=False)
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')